In [1]:
from google.colab import files
uploaded = files.upload()

Saving reviews_data_dump.zip to reviews_data_dump.zip


In [2]:
!unzip reviews_data_dump.zip

Archive:  reviews_data_dump.zip
  inflating: reviews_badminton/credits.txt  
  inflating: reviews_badminton/data.csv  
  inflating: reviews_tawa/credits.txt  
  inflating: reviews_tawa/data.csv   
  inflating: reviews_tea/credits.txt  
  inflating: reviews_tea/data.csv    


In [4]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
df_badminton = pd.read_csv('reviews_badminton/data.csv')

In [6]:
df_badminton.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
column_name = 'Review text'

In [10]:
def clean_text(text):
    text = re.sub(r'\W', ' ', text)


    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)


    text = re.sub(r'\s+', ' ', text, flags=re.I)

    text = text.lower()

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if not word in stop_words]
    text = ' '.join(filtered_text)

    return text

In [11]:
df_badminton[column_name] = df_badminton[column_name].apply(lambda x: clean_text(str(x)))

In [12]:
df_badminton.to_csv('cleaned_reviews.csv', index=False)

In [13]:
df2 = pd.read_csv('cleaned_reviews.csv')

In [14]:
df2.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,nice product good quality price rising bad sig...,4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,supplied yonex mavis 350 outside cover yonex a...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,worst product damaged shuttlecocks packed new ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,quite k nowadays quality corks like 3 5 years ...,3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,pricedjust â ¹620 retailer understand wat adva...,1


In [15]:
def clean_and_normalize_text(text):

    text = re.sub(r'\W', ' ', text)

    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)

    text = re.sub(r'\s+', ' ', text, flags=re.I)


    text = text.lower()


    word_tokens = word_tokenize(text)


    stop_words = set(stopwords.words('english'))
    filtered_text = [word for word in word_tokens if word not in stop_words]

    porter = PorterStemmer()
    stemmed_text = [porter.stem(word) for word in filtered_text]

    return ' '.join(stemmed_text)

In [16]:
df_badminton[column_name] = df_badminton[column_name].apply(lambda x: clean_and_normalize_text(str(x)))

In [17]:
df_badminton.to_csv('cleaned_normalized_reviews.csv', index=False)

In [18]:
df3 = pd.read_csv('cleaned_normalized_reviews.csv')

In [19]:
df3.shape

(8518, 8)

In [20]:
df3 = df3.dropna()

In [21]:
df3.shape

(8013, 8)

In [22]:
df3.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,nice product good qualiti price rise bad sign ...,4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,suppli yonex mavi 350 outsid cover yonex ad in...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,worst product damag shuttlecock pack new box o...,1
5,Baji Sankar,Mind-blowing purchase,"Certified Buyer, Hyderabad",173.0,45.0,Oct 2018,good qualiti product deliv time read,5
6,Flipkart Customer,Must buy!,"Certified Buyer, Doom Dooma",403.0,121.0,Jan 2020,best purchas good qualiti durabl averag shuttl...,5


Numerical Feature Extraction
Bag-of-Words(BoW)

In [23]:
corpus = df3['Review text'].tolist()

# Bag-of-Words (BoW)
count_vectorizer = CountVectorizer()
X_bow = count_vectorizer.fit_transform(corpus)

# Term Frequency-Inverse Document Frequency (TF-IDF)
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(corpus)

# Save feature matrices to new CSV files
pd.DataFrame(X_bow.toarray(), columns=count_vectorizer.get_feature_names_out()).to_csv('bow_features.csv', index=False)
pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out()).to_csv('tfidf_features.csv', index=False)

Word2Vec (W2V)

In [24]:
corpus = [nltk.word_tokenize(text) for text in df3['Review text'].tolist()]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)

# Save Word2Vec model
word2vec_model.save('word2vec_model.model')

In [25]:
word2vec_embeddings = []

for line in corpus:
  embedding = []
  for word in line:
    try:
      embd = word2vec_model.wv[word]
      embedding.append(embd)
    except:
      continue
  embedding = np.mean(np.array(embedding), axis=0)
  word2vec_embeddings.append(embedding)
pickle.dump(np.array(word2vec_embeddings), open('word2vec_embeddings.pickle', 'wb'))

BERT

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.to('cuda')


corpus = df3['Review text'].tolist()
batch = 100
embeddings = []

for i in range(0, len(corpus), batch):

  inputs = tokenizer(corpus[i:i+batch], return_tensors='pt', padding='max_length', max_length=100, truncation=True)
  inputs.to('cuda')

  with torch.no_grad():
      outputs = model(**inputs)

  bert_embeddings = outputs.last_hidden_state.cpu()
  embeddings.append(bert_embeddings)

embeddings = np.vstack(embeddings)

pickle.dump(embeddings, open('bert_embeddings.pickle', 'wb'))

print("BERT embeddings extracted and saved to 'bert_embeddings.pickle'")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT embeddings extracted and saved to 'bert_embeddings.pickle'


In [27]:

data = pd.read_csv('bow_features.csv')

X = data
y = df3['Ratings'].apply(lambda x : 1 if x > 3 else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print(f1_score(y_test, y_pred))

models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': make_pipeline(StandardScaler(), SVC())
}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    print(f"Evaluating {name}...")
    evaluate_model(model, X_test, y_test)

max_words = X_train.shape[1]
embedding_dim = 100
max_length = 100


model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_length),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


X_train_seq = pad_sequences(X_train.to_numpy(), maxlen=max_length)
X_test_seq = pad_sequences(X_test.to_numpy(), maxlen=max_length)

early_stopping = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)

print("Training deep learning model...")
model.fit(X_train_seq, y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[early_stopping])


print("Evaluating deep learning model...")
evaluate_model(model, X_test_seq, y_test)


Training Logistic Regression...
Evaluating Logistic Regression...
0.9336791404223788
Training Random Forest...
Evaluating Random Forest...
0.930648769574944
Training SVM...
Evaluating SVM...
0.9142651296829972
Training deep learning model...
Epoch 1/10
181/181 [==============================] - 4s 7ms/step - loss: 0.4926 - accuracy: 0.8053 - val_loss: 0.5187 - val_accuracy: 0.7956
Epoch 2/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4752 - accuracy: 0.8187 - val_loss: 0.4991 - val_accuracy: 0.8003
Epoch 3/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4717 - accuracy: 0.8215 - val_loss: 0.4962 - val_accuracy: 0.8050
Epoch 4/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4679 - accuracy: 0.8222 - val_loss: 0.4978 - val_accuracy: 0.8066
Epoch 5/10
181/181 [==============================] - 1s 5ms/step - loss: 0.4638 - accuracy: 0.8235 - val_loss: 0.5050 - val_accuracy: 0.8066
Epoch 6/10
181/181 [============================

In [28]:

data = pd.read_csv('tfidf_features.csv')

X = data
y = df3['Ratings'].apply(lambda x : 1 if x > 3 else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print(f1_score(y_test, y_pred))


models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': make_pipeline(StandardScaler(), SVC())
}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    print(f"Evaluating {name}...")
    evaluate_model(model, X_test, y_test)


max_words = X_train.shape[1]
embedding_dim = 100
max_length = 100

model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_length),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


X_train_seq = pad_sequences(X_train.to_numpy(), maxlen=max_length)
X_test_seq = pad_sequences(X_test.to_numpy(), maxlen=max_length)


early_stopping = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)

print("Training deep learning model...")
model.fit(X_train_seq, y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[early_stopping])


print("Evaluating deep learning model...")
evaluate_model(model, X_test_seq, y_test)


Training Logistic Regression...
Evaluating Logistic Regression...
0.9301134284668862
Training Random Forest...
Evaluating Random Forest...
0.9284390063033
Training SVM...
Evaluating SVM...
0.9165763813651137
Training deep learning model...
Epoch 1/10
181/181 [==============================] - 2s 6ms/step - loss: 0.4957 - accuracy: 0.8088 - val_loss: 0.5083 - val_accuracy: 0.7956
Epoch 2/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4929 - accuracy: 0.8088 - val_loss: 0.5251 - val_accuracy: 0.7956
Epoch 3/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4898 - accuracy: 0.8088 - val_loss: 0.5054 - val_accuracy: 0.7956
Epoch 4/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4922 - accuracy: 0.8093 - val_loss: 0.5109 - val_accuracy: 0.7972
Epoch 5/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4878 - accuracy: 0.8102 - val_loss: 0.5104 - val_accuracy: 0.7972
Epoch 6/10
181/181 [==============================

In [29]:

data = pickle.load(open('word2vec_embeddings.pickle','rb'))


X = pd.DataFrame(data)
y = df3['Ratings'].apply(lambda x : 1 if x > 3 else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print(f1_score(y_test, y_pred))

models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': make_pipeline(StandardScaler(), SVC())
}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    print(f"Evaluating {name}...")
    evaluate_model(model, X_test, y_test)

max_words = X_train.shape[1]
embedding_dim = 100
max_length = 100

model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_length),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X_train_seq = pad_sequences(X_train.to_numpy(), maxlen=max_length)
X_test_seq = pad_sequences(X_test.to_numpy(), maxlen=max_length)

early_stopping = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)

print("Training deep learning model...")
model.fit(X_train_seq, y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

print("Evaluating deep learning model...")
evaluate_model(model, X_test_seq, y_test)


Training Logistic Regression...
Evaluating Logistic Regression...
0.8902734510211145
Training Random Forest...
Evaluating Random Forest...
0.903971119133574
Training SVM...
Evaluating SVM...
0.893528183716075
Training deep learning model...
Epoch 1/10
181/181 [==============================] - 2s 7ms/step - loss: 0.4967 - accuracy: 0.8088 - val_loss: 0.5079 - val_accuracy: 0.7956
Epoch 2/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4924 - accuracy: 0.8088 - val_loss: 0.5066 - val_accuracy: 0.7956
Epoch 3/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4920 - accuracy: 0.8088 - val_loss: 0.5093 - val_accuracy: 0.7956
Epoch 4/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4932 - accuracy: 0.8088 - val_loss: 0.5193 - val_accuracy: 0.7956
Epoch 5/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4917 - accuracy: 0.8088 - val_loss: 0.5091 - val_accuracy: 0.7956
Evaluating deep learning model...
51/51 [========

In [30]:
data = pickle.load(open('bert_embeddings.pickle','rb'))

X = pd.DataFrame(np.mean(data, axis=1))
y = df3['Ratings'].apply(lambda x : 1 if x > 3 else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print(f1_score(y_test, y_pred))

models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': make_pipeline(StandardScaler(), SVC())
}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    print(f"Evaluating {name}...")
    evaluate_model(model, X_test, y_test)

max_words = X_train.shape[1]
embedding_dim = 100
max_length = 100

model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_length),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X_train_seq = pad_sequences(X_train.to_numpy(), maxlen=max_length)
X_test_seq = pad_sequences(X_test.to_numpy(), maxlen=max_length)

early_stopping = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)

print("Training deep learning model...")
model.fit(X_train_seq, y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

print("Evaluating deep learning model...")
evaluate_model(model, X_test_seq, y_test)


Training Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Evaluating Logistic Regression...
0.926078799249531
Training Random Forest...
Evaluating Random Forest...
0.9222873900293255
Training SVM...
Evaluating SVM...
0.9303187546330615
Training deep learning model...
Epoch 1/10
181/181 [==============================] - 3s 11ms/step - loss: 0.4939 - accuracy: 0.8060 - val_loss: 0.5057 - val_accuracy: 0.7956
Epoch 2/10
181/181 [==============================] - 1s 7ms/step - loss: 0.4923 - accuracy: 0.8088 - val_loss: 0.5073 - val_accuracy: 0.7956
Epoch 3/10
181/181 [==============================] - 1s 8ms/step - loss: 0.4950 - accuracy: 0.8088 - val_loss: 0.5065 - val_accuracy: 0.7956
Epoch 4/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4908 - accuracy: 0.8088 - val_loss: 0.5048 - val_accuracy: 0.7956
Epoch 5/10
181/181 [==============================] - 1s 4ms/step - loss: 0.4891 - accuracy: 0.8088 - val_loss: 0.5157 - val_accuracy: 0.7956
Epoch 6/10
181/181 [==============================] - 1s 4ms/step - loss: 0.490